In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from ase.io import read,write
from ase.visualize import view
import numpy as np 

from equistore import Labels, TensorBlock, TensorMap
from itertools import product
from equistore_utils.clebsh_gordan import ClebschGordanReal

import matplotlib.pyplot as plt

# from equistore_utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
# from rascal.representations import SphericalExpansion
from equistore_utils.acdc_mini import acdc_standardize_keys, _remove_suffix
from rascaline import SphericalExpansion
from rascaline import SphericalExpansionByPair as PairExpansion
from equistore import operations
import copy
import ase 
import tqdm 
import hickle

In [3]:
from equistore_utils.mp_utils import *

In [4]:
from ase.io import read

In [144]:
# frames_all = read('bispectrum_counter/carbon/C-VII-pp-wrapped.xyz', ':10000')
# leng = np.asarray([len(f) for f in frames_all])
# frames_14 = []
# for idx in np.where(leng<14)[0]:
# #     print(idx)
#     frames_14.append(frames_all[idx])
# frames = frames_14
# write('bispectrum_counter/carbon/C_lessthan14.xyz', frames)
# print(np.max([np.max(f.get_all_distances()) for f in frames_14]))
frames = read('/home/nigam/scratch/MAC/bispectrum_counter/carbon/C-VII-pp-wrapped.xyz', ':')
print(len(frames))
# frames = read('bisp/C_lessthan14.xyz', ':10')

10874


In [80]:
frames[0].info

{'pressure': 10.0029,
 'name': 'C                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               -                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Set up basic features

In [6]:
from rascaline import SphericalExpansion
from rascaline import SphericalExpansionByPair as PairExpansion

In [7]:
hypers = {
    "cutoff": 3,
    "max_radial": 4,
    "max_angular": 3,
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.1}},
    "center_atom_weight": 0.0,    
}

hypers_ij = {
    "cutoff": 2,
    "max_radial": 4,
    "max_angular": 3,
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.1}},
    "center_atom_weight": 0.0,    
}

cg = ClebschGordanReal(5)


In [8]:
calculator = SphericalExpansion(**hypers)
rhoi = calculator.compute(frames[:])
rhoi = rhoi.keys_to_properties(['species_neighbor'])
rho1i = acdc_standardize_keys(rhoi)
# rho1i = operations.slice(rho1i, samples=Labels(['center'],np.array([[0]], np.int32)) )
# norm_rho1 = np.sqrt(np.sum([(b.values**2).sum(axis=(1,2)) for b in rho1i.blocks()],axis=0).mean())
# for b in rho1i.blocks():
#     b.values[:]/=norm_rho1
    

In [9]:
rho2i = cg_combine(rho1i, rho1i, clebsch_gordan=cg, other_keys_match=['species_center'], lcut=3)

In [10]:
rho3i =  cg_combine(rho2i, rho1i, clebsch_gordan=cg, other_keys_match=['species_center'], lcut=0)

In [11]:
calculator = PairExpansion(**hypers_ij)
gij = calculator.compute(frames[:])
# gij = operations.slice(gij, samples=Labels(['first_atom'],np.array([[0]], np.int32)) )
gij =  acdc_standardize_keys(gij)
# for b in gij.blocks():
# #     b.values[:]/=norm_rho1
# # gij = simplify_gij(gij)

In [12]:
# test = operations.sum_over_samples(gij, samples_names=["neighbor"])#, "pair_id"]) 
# np.linalg.norm(rho1i.block(0).values/test.block(0).values -1)

## Compute three-feats

In [13]:
rhoii1i2_nu0 = cg_combine(gij, gij, clebsch_gordan=cg, other_keys_match=['species_center'], lcut=3)

In [14]:
rhoii1i2_nu1 =  cg_combine(rho1i, rhoii1i2_nu0, clebsch_gordan=cg, other_keys_match = ['species_center'], lcut=0)

In [20]:
from sklearn.decomposition import PCA

raw = np.hstack([rhoii1i2_nu1.block(0).values.squeeze(),rhoii1i2_nu1.block(1).values.squeeze()])
print(raw.shape)
feats_n2nu1 = PCA(n_components=min(raw.shape[0],raw.shape[-1])).fit_transform(raw)
# hickle.dump (raw, feat_file+ 'feat_3cnu1.hickle')
# hickle.dump(feats_n2nu1, feat_file+'feat_3cnu1_PCA.hickle')

(17276, 2176)


## Set up energy model 

In [145]:
import torch 
device='cpu'
torch.set_default_dtype(torch.float64)

energy = torch.tensor([f.info["pp_energy"] for f in frames], device=device)
print(torch.min(energy), torch.max(energy))

tensor(-154.5565) tensor(-150.9199)


In [153]:
from ase.units import kcal, Hartree
from ase.units import mol
hartree_kcalpmol = 627.503


0.04336410390059322

In [155]:
kcalpmol/Hartree

23.060310101002713

In [156]:
torch.std(energy)*kcalpmol/Hartree

tensor(10.7209)

In [129]:
def StructureMap(samples_structure, device="cpu"):
    unique_structures, unique_structures_idx = np.unique(
        samples_structure, return_index=True
    )   
    new_samples = samples_structure[np.sort(unique_structures_idx)]
    # we need a list keeping track of where each atomic contribution goes
    # (e.g. if structure ids are [3,3,3,1,1,1,6,6,6] that will be stored as
    # the unique structures [3, 1, 6], structure_map will be
    # [0,0,0,1,1,1,2,2,2]
    replace_rule = dict(zip(unique_structures, range(len(unique_structures))))
    structure_map = torch.tensor(
        [replace_rule[i] for i in samples_structure],
        dtype=torch.long,
        device=device,
    )   
    return structure_map, new_samples, replace_rule

In [130]:
class TripletEnergy(torch.nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.input_size = kwargs["input_size"]
        self.intermediate_size = kwargs["intermediate_size"]
        self.hidden_size = kwargs["hidden_size"]
        self.input_layer = torch.nn.Linear(
            in_features=self.input_size, out_features=self.intermediate_size)

        self.hidden_layer = torch.nn.Linear(
                in_features=self.intermediate_size, out_features=self.hidden_size
            )

        self.out_layer = torch.nn.Linear(
                in_features=self.hidden_size, out_features=1
            )

        self.model = torch.nn.Sequential( self.input_layer,
                                        torch.nn.GroupNorm(4,self.intermediate_size),
                                        torch.nn.SiLU(),
                                         self.hidden_layer,
                                         torch.nn.GroupNorm(4,self.hidden_size),
                                         torch.nn.SiLU(),
                                         self.out_layer
            )

        self.sum_triples=None
#         self.samples = kwargs["triple_samples"]
        self.to(device)

    def structure_wise_energy(self, x, samples):

        structure_map, new_samples, _ = StructureMap(
            samples["structure"], "cpu"
        )
#         print(encoded.shape)
        self.sum_triples =  torch.zeros((len(new_samples), x.shape[-1]), device=x.device)
        self.sum_triples.index_add_(0, structure_map, x)

        return self.sum_triples

    def forward(self, x, samples):

        pred = self.model(x)
        pred = self.structure_wise_energy(pred, samples)

        return pred


# frames = read(frames_file, ':3000')
triple_sample_names = rhoii1i2_nu1.sample_names#np.load(feat_file + 'triple_samples_names.npy')
triple_sample_array = rhoii1i2_nu1[0].samples.view(np.int32).copy().reshape(-1, len(triple_sample_names) )#np.load(feat_file + 'triple_samples.npy')

triple_samples = rhoii1i2_nu1[0].samples #Labels(list(triple_sample_names), triple_sample_array)


In [138]:
feats_n2nu1 = torch.tensor(feats_n2nu1, device=device)

triple_enmodel = TripletEnergy(input_size = feats_n2nu1.shape[-1],
                               intermediate_size = 512,
                               hidden_size = 128,)


/tmp/ipykernel_22838/2043345322.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feats_n2nu1 = torch.tensor(feats_n2nu1, device=device)


In [139]:
def get_triple_sample_idx(i,triple_sample_array):
    #return first instance of structure i
    try:
        return np.where(triple_sample_array[:,0]==i)[0][0]
    except: 
        return None
    
def iterate_minibatches(inputs, outputs, batch_size, triple_sample_array):
    nstruct = triple_sample_array[-1][0]
    print("number of structures found ", nstruct)
    for index in range(0, nstruct , batch_size):
#         if index+batc
        start = get_triple_sample_idx(index,triple_sample_array)
        stop = get_triple_sample_idx(index+batch_size,triple_sample_array)
#         print(index, start, stop)
        yield inputs[start : stop], outputs[index : index + batch_size], index
        


In [140]:
def mse_loss(pred, target):
    return torch.sum((pred.flatten() - target.flatten()) ** 2)
best = 1e10
energy_optimizer = torch.optim.Adam(
        triple_enmodel.parameters(),
        lr=1e-3,
        #line_search_fn="strong_wolfe",
        #history_size=128
)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(energy_optimizer, factor = 0.8, patience = 200)


In [141]:
triple_sample_train = triple_sample_array[:get_triple_sample_idx(ntrain,triple_sample_array)].copy()
triple_sample_test = triple_sample_array[get_triple_sample_idx(ntrain,triple_sample_array):].copy()
triple_sample_test[:,0]-= ntrain

In [142]:
triple_sample_train

Labels([[  0,   0, -11, -11],
        [  0,   0, -11,   2],
        [  0,   0, -11,  14],
        ...,
        [ 69,   3,  57,  22],
        [ 69,   3,  57,  45],
        [ 69,   3,  57,  57]], dtype=int32)

In [143]:
ntrain=70
BATCH_SIZE = 10
n_epochs = 500

for epoch in range(0,  n_epochs):
    all_predictions = []
    triple_enmodel.train(True)
    for feat, target, index in iterate_minibatches(feats_n2nu1[:get_triple_sample_idx(ntrain,triple_sample_array)], energy[:ntrain], BATCH_SIZE,triple_sample_train):
        start = get_triple_sample_idx(index,triple_sample_train)
        stop = get_triple_sample_idx(index+BATCH_SIZE,triple_sample_train)
#         print(start,stop)
        samples= Labels(triple_sample_names, triple_sample_train[start:stop])
#         print(samples)
        predicted = triple_enmodel.forward(feat, samples)
#         print("here")
#         print(index,samples)
        all_predictions.append(predicted.data.detach().numpy())
        l = mse_loss(predicted, target)
        l.backward()
        energy_optimizer.step()
        energy_optimizer.zero_grad()

    all_predictions = np.concatenate(all_predictions, axis = 0)
#     print(all_predictions.shape)
    train_error = mse_loss(torch.tensor(all_predictions), energy[:ntrain])
    scheduler.step(train_error)
    
    triple_enmodel.train(False)
    test_all_predictions = []
    for feat, target,index in iterate_minibatches(feats_n2nu1[get_triple_sample_idx(ntrain,triple_sample_array):], energy[ntrain:], BATCH_SIZE,triple_sample_test):
        start = get_triple_sample_idx(index,triple_sample_test)
        stop = get_triple_sample_idx(index+BATCH_SIZE,triple_sample_test)
#         print(start,stop)
        tsamples= Labels(triple_sample_names, triple_sample_test[start:stop])
#         print(tsamples)
        predicted = triple_enmodel.forward(feat, tsamples)
#         print("here")
        test_all_predictions.append(predicted.data.detach().numpy())

    test_all_predictions = np.concatenate(test_all_predictions, axis = 0)
#     print(test_all_predictions.shape)
    val_error = mse_loss(torch.tensor(test_all_predictions), energy[ntrain:])

    print("epoch: ", epoch, "RMSE train: ", np.sqrt(train_error.detach().numpy()/(ntrain)),
          "RMSE test: ", np.sqrt(val_error.detach().numpy()/((len(frames)-ntrain))))

number of structures found  69
number of structures found  29
epoch:  0 RMSE train:  122.53702799680387 RMSE test:  94.0647690597713
number of structures found  69
number of structures found  29
epoch:  1 RMSE train:  74.72397140156343 RMSE test:  62.411488582078285
number of structures found  69
number of structures found  29
epoch:  2 RMSE train:  65.57199482887881 RMSE test:  61.08341370060381
number of structures found  69
number of structures found  29
epoch:  3 RMSE train:  56.59137201070312 RMSE test:  57.02981061665817
number of structures found  69
number of structures found  29
epoch:  4 RMSE train:  59.34319741349338 RMSE test:  55.86443690256707
number of structures found  69
number of structures found  29
epoch:  5 RMSE train:  58.08335142924679 RMSE test:  57.06336683654979
number of structures found  69
number of structures found  29
epoch:  6 RMSE train:  57.249557471198685 RMSE test:  55.89765888442205
number of structures found  69
number of structures found  29
epoch

number of structures found  29
epoch:  61 RMSE train:  32.967151203701384 RMSE test:  50.34160869355067
number of structures found  69
number of structures found  29
epoch:  62 RMSE train:  32.16234680675262 RMSE test:  51.4194546560129
number of structures found  69
number of structures found  29
epoch:  63 RMSE train:  31.535532070118034 RMSE test:  51.87839663492831
number of structures found  69
number of structures found  29
epoch:  64 RMSE train:  31.063392943506756 RMSE test:  51.852324305842984
number of structures found  69
number of structures found  29
epoch:  65 RMSE train:  30.867903469548093 RMSE test:  51.95655341928724
number of structures found  69
number of structures found  29
epoch:  66 RMSE train:  30.592149406585104 RMSE test:  51.52554507376946
number of structures found  69
number of structures found  29
epoch:  67 RMSE train:  30.87146653110275 RMSE test:  51.45221007082974
number of structures found  69
number of structures found  29
epoch:  68 RMSE train:  30

number of structures found  29
epoch:  122 RMSE train:  20.247731734979933 RMSE test:  41.336033750132465
number of structures found  69
number of structures found  29
epoch:  123 RMSE train:  19.795722122961582 RMSE test:  46.06300952260435
number of structures found  69
number of structures found  29
epoch:  124 RMSE train:  21.199415559707592 RMSE test:  48.274003682858
number of structures found  69
number of structures found  29
epoch:  125 RMSE train:  20.009107067960187 RMSE test:  47.817718854076574
number of structures found  69
number of structures found  29
epoch:  126 RMSE train:  18.59632465216784 RMSE test:  40.49024690436227
number of structures found  69
number of structures found  29
epoch:  127 RMSE train:  18.702958884019015 RMSE test:  38.990781562844376
number of structures found  69
number of structures found  29
epoch:  128 RMSE train:  18.722077043051264 RMSE test:  37.44467611542986
number of structures found  69
number of structures found  29
epoch:  129 RMSE 

number of structures found  29
epoch:  183 RMSE train:  14.482509148818886 RMSE test:  44.21823462321802
number of structures found  69
number of structures found  29
epoch:  184 RMSE train:  14.16065576572478 RMSE test:  38.4420323079714
number of structures found  69
number of structures found  29
epoch:  185 RMSE train:  13.94602326829754 RMSE test:  36.72823618356185
number of structures found  69
number of structures found  29
epoch:  186 RMSE train:  13.482728888019668 RMSE test:  34.87909012889844
number of structures found  69
number of structures found  29
epoch:  187 RMSE train:  13.54330035957007 RMSE test:  35.183841778550715
number of structures found  69
number of structures found  29
epoch:  188 RMSE train:  14.099267932877376 RMSE test:  35.35839328920768
number of structures found  69
number of structures found  29
epoch:  189 RMSE train:  14.291853424641747 RMSE test:  39.25028186892826
number of structures found  69
number of structures found  29
epoch:  190 RMSE tra

number of structures found  29
epoch:  244 RMSE train:  10.745203041482458 RMSE test:  36.73734221906181
number of structures found  69
number of structures found  29
epoch:  245 RMSE train:  10.383648095995607 RMSE test:  35.04009287411252
number of structures found  69
number of structures found  29
epoch:  246 RMSE train:  10.721086580037854 RMSE test:  34.79338548922117
number of structures found  69
number of structures found  29
epoch:  247 RMSE train:  11.420475311748472 RMSE test:  35.86883006133334
number of structures found  69
number of structures found  29
epoch:  248 RMSE train:  11.604383120522606 RMSE test:  39.46193410304842
number of structures found  69
number of structures found  29
epoch:  249 RMSE train:  11.468900365340655 RMSE test:  42.76538848596283
number of structures found  69
number of structures found  29
epoch:  250 RMSE train:  11.367286841779185 RMSE test:  38.409563712247234
number of structures found  69
number of structures found  29
epoch:  251 RMSE

number of structures found  29
epoch:  305 RMSE train:  11.044107154851089 RMSE test:  42.73389396297401
number of structures found  69
number of structures found  29
epoch:  306 RMSE train:  10.329350625804485 RMSE test:  37.3157608619876
number of structures found  69
number of structures found  29
epoch:  307 RMSE train:  10.305342950893978 RMSE test:  38.096545915153804
number of structures found  69
number of structures found  29
epoch:  308 RMSE train:  9.362253328797616 RMSE test:  34.84667498379463
number of structures found  69
number of structures found  29
epoch:  309 RMSE train:  9.965250806068639 RMSE test:  34.82552861507269
number of structures found  69
number of structures found  29
epoch:  310 RMSE train:  10.148153650020543 RMSE test:  35.58527542170523
number of structures found  69
number of structures found  29
epoch:  311 RMSE train:  10.665144339728302 RMSE test:  38.412333361490774
number of structures found  69
number of structures found  29
epoch:  312 RMSE t

number of structures found  29
epoch:  366 RMSE train:  9.641516608258224 RMSE test:  35.46784803675415
number of structures found  69
number of structures found  29
epoch:  367 RMSE train:  9.986755644538995 RMSE test:  35.468061144263814
number of structures found  69
number of structures found  29
epoch:  368 RMSE train:  10.148818027115091 RMSE test:  37.2403571311208
number of structures found  69
number of structures found  29
epoch:  369 RMSE train:  10.154337260128619 RMSE test:  39.525561350365216
number of structures found  69
number of structures found  29
epoch:  370 RMSE train:  10.116092077265913 RMSE test:  40.61957185689914
number of structures found  69
number of structures found  29
epoch:  371 RMSE train:  9.683199527574722 RMSE test:  38.754288737832006
number of structures found  69
number of structures found  29
epoch:  372 RMSE train:  9.311712878850438 RMSE test:  37.18101815628265
number of structures found  69
number of structures found  29
epoch:  373 RMSE tr

number of structures found  29
epoch:  427 RMSE train:  9.907831658249487 RMSE test:  37.061885529538806
number of structures found  69
number of structures found  29
epoch:  428 RMSE train:  10.453741600085117 RMSE test:  41.88108649267235
number of structures found  69
number of structures found  29
epoch:  429 RMSE train:  9.87023459496415 RMSE test:  38.69147471320973
number of structures found  69
number of structures found  29
epoch:  430 RMSE train:  9.876334627145917 RMSE test:  41.67349666047647
number of structures found  69
number of structures found  29
epoch:  431 RMSE train:  9.342683466381043 RMSE test:  35.26996811342047
number of structures found  69
number of structures found  29
epoch:  432 RMSE train:  10.118985658342691 RMSE test:  36.714357249222324
number of structures found  69
number of structures found  29
epoch:  433 RMSE train:  9.327846965743783 RMSE test:  34.63781949166282
number of structures found  69
number of structures found  29
epoch:  434 RMSE trai

number of structures found  29
epoch:  488 RMSE train:  12.214358810744551 RMSE test:  43.809819134157664
number of structures found  69
number of structures found  29
epoch:  489 RMSE train:  10.688818898917464 RMSE test:  38.769261955908235
number of structures found  69
number of structures found  29
epoch:  490 RMSE train:  9.821732960560743 RMSE test:  34.449735283643875
number of structures found  69
number of structures found  29
epoch:  491 RMSE train:  9.697649426127374 RMSE test:  34.143081300450184
number of structures found  69
number of structures found  29
epoch:  492 RMSE train:  9.827356932535727 RMSE test:  38.14911129799643
number of structures found  69
number of structures found  29
epoch:  493 RMSE train:  9.46133663021205 RMSE test:  41.18117137330455
number of structures found  69
number of structures found  29
epoch:  494 RMSE train:  9.268384421189735 RMSE test:  40.219988139593575
number of structures found  69
number of structures found  29
epoch:  495 RMSE t

In [137]:
fnat = np.asarray([len(f) for f in frames])

In [32]:
sum(fnat**3)

72208

In [45]:
triple_sample_array[np.where(triple_sample_array[:,0]==20)[0]]

Labels([[ 20,   0, -11, -11],
        [ 20,   0, -11,   1],
        [ 20,   0, -11,  15],
        [ 20,   0, -11,  27],
        [ 20,   0, -11,  40],
        [ 20,   1, -10, -10],
        [ 20,   1, -10,   0],
        [ 20,   1, -10,  48],
        [ 20,   1, -10,  60],
        [ 20,   1, -10,  72],
        [ 20,   2,  -9,  -9],
        [ 20,   2,  -9,  80],
        [ 20,   2,  -9,  91],
        [ 20,   2,  -9, 106],
        [ 20,   3,  -8,  -8],
        [ 20,   3,  -8,  79],
        [ 20,   3,  -8,  90],
        [ 20,   3,  -8, 116],
        [ 20,   4,  -7,  -7],
        [ 20,   4,  -7,  11],
        [ 20,   4,  -7,  45],
        [ 20,   4,  -7, 126],
        [ 20,   4,  -7, 137],
        [ 20,   5,  -6,  -6],
        [ 20,   5,  -6,  22],
        [ 20,   5,  -6,  56],
        [ 20,   5,  -6, 125],
        [ 20,   5,  -6, 136],
        [ 20,   6,  -5,  -5],
        [ 20,   6,  -5,  67],
        [ 20,   6,  -5, 113],
        [ 20,   6,  -5, 150],
        [ 20,   6,  -5, 161],
        [ 

In [38]:
np.where(triple_sample_array[:,0]==1)[0]

array([214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252,
       253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
       266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278,
       279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291,
       292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304,
       305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317,
       318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330,
       331])